# Animierte Mesh-Vergleiche
Dieses Notebook erstellt rotierende GIF-Animationen der verschiedenen 3D-Rekonstruktionsmethoden.

In [1]:
import pandas as pd
import numpy as np
import trimesh
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import os
from pathlib import Path
import sys
import fast_simplification

sys.path.append("../")
from TestEvaluationPipeline.mesh_utils import MeshUtils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def find_mesh_file(model_dir, base_name, exts=(".stl", ".obj")):
    """
    Look for a file in model_dir whose name contains base_name and ends with one of the given extensions.
    """
    base = Path(model_dir)
    # try scanning for any file matching pattern *base_name*ext
    for ext in exts:
        pattern = f"*{base_name}*{ext}"
        matches = list(base.glob(pattern))
        if matches:
            return str(matches[0])
    # fallback to any file containing base_name
    globbed = list(base.glob(f"*{base_name}*.*"))
    if globbed:
        return str(globbed[0])
    raise FileNotFoundError(f"No mesh file containing '{base_name}' in '{model_dir}'")


def set_axes_equal_and_zoom(ax, mesh, zoom=1.0):
    xyz = mesh.vertices
    mins = xyz.min(axis=0)
    maxs = xyz.max(axis=0)
    center = (mins + maxs) / 2
    half = (maxs - mins).max() * zoom / 2

    ax.set_xlim(center[0] - half, center[0] + half)
    ax.set_ylim(center[1] - half, center[1] + half)
    ax.set_zlim(center[2] - half, center[2] + half)
    ax.set_box_aspect((1, 1, 1))

In [ ]:
def create_animated_mesh_comparison(
    basenames,
    model_dirs,
    model_names=None,
    exts=(".stl", ".obj"),
    zoom=0.8,
    figsize=(15, 10),
    frames=36,  # Anzahl der Frames für 360° Rotation
    fps=6,  # Frames per second
    output_file="mesh_comparison.gif",
    colormap_name="viridis",  # viridis, plasma, coolwarm, magma
):
    """
    Erstellt eine animierte GIF-Datei mit rotierenden Meshes.

    basenames       : list of mesh name substrings WITHOUT extension
    model_dirs      : list of directory paths (first one = GT)
    model_names     : optional list of column titles
    exts            : tuple of extensions to try (default .stl and .obj)
    zoom            : <1 zooms in (meshes look larger), >1 zooms out
    figsize         : figure size for plt.figure
    frames          : number of animation frames (36 = 10° per frame)
    fps             : frames per second for the GIF
    output_file     : filename for the output GIF
    colormap_name   : matplotlib colormap name for depth coloring
    """
    n_rows = len(basenames)
    n_cols = len(model_dirs)

    if model_names is None:
        model_names = [os.path.basename(os.path.normpath(d)) for d in model_dirs]

    # Lade und verarbeite alle Meshes einmal
    meshes = {}
    for i, name in enumerate(basenames):
        # load GT for this row
        gt_path = find_mesh_file(model_dirs[0], name, exts)
        gt = trimesh.load(gt_path)
        meshes[(i, 0)] = gt

        for j, model_dir in enumerate(model_dirs[1:], 1):
            pred_path = find_mesh_file(model_dir, name, exts)
            pred = trimesh.load(pred_path)

            # Simplify if too complex - Special handling for different models
            model_name = os.path.basename(model_dir)

            if "hunyuan" in model_name.lower():
                # Hunyuan3D produces very high-res meshes - be more conservative
                if pred.vertices.shape[0] >= 20_000:
                    target_vertices = min(
                        8_000, pred.vertices.shape[0] // 2
                    )  # Keep more detail
                    reduction_ratio = 1 - (target_vertices / pred.vertices.shape[0])
                    reduction_ratio = max(
                        0.2, min(0.7, reduction_ratio)
                    )  # 20-70% reduction

                    new_verts, new_faces = fast_simplification.simplify(
                        points=pred.vertices.view(np.ndarray),
                        triangles=pred.faces.view(np.ndarray),
                        target_reduction=reduction_ratio,
                    )
                    pred = trimesh.Trimesh(vertices=new_verts, faces=new_faces)
                    print(
                        f"Hunyuan3D: Simplified from {pred.vertices.shape[0]} to {new_verts.shape[0]} vertices (conservative)"
                    )
            else:
                # Other models - standard aggressive simplification for animation
                if pred.vertices.shape[0] >= 5_000:
                    target_vertices = min(3_000, pred.vertices.shape[0] // 3)
                    reduction_ratio = 1 - (target_vertices / pred.vertices.shape[0])
                    reduction_ratio = max(
                        0.5, min(0.95, reduction_ratio)
                    )  # 50-95% reduction

                    new_verts, new_faces = fast_simplification.simplify(
                        points=pred.vertices.view(np.ndarray),
                        triangles=pred.faces.view(np.ndarray),
                        target_reduction=reduction_ratio,
                    )
                    pred = trimesh.Trimesh(vertices=new_verts, faces=new_faces)
                    print(
                        f"{model_name}: Simplified from {pred.vertices.shape[0]} to {new_verts.shape[0]} vertices"
                    )

            # Align to GT
            aligned, _ = MeshUtils.align_icp(pred, gt)
            mesh = trimesh.Trimesh(vertices=aligned.vertices, faces=pred.faces)
            meshes[(i, j)] = mesh

    # Erstelle die Animation
    fig, axes = plt.subplots(
        n_rows, n_cols, subplot_kw={"projection": "3d"}, figsize=figsize, squeeze=False
    )

    # Setup axes styling
    plots = {}
    for i in range(n_rows):
        for j in range(n_cols):
            ax = axes[i, j]
            ax.grid(False)
            ax.xaxis.pane.fill = False
            ax.yaxis.pane.fill = False
            ax.zaxis.pane.fill = False
            for spine in (ax.xaxis, ax.yaxis, ax.zaxis):
                spine.pane.set_edgecolor("none")
            ax.set_axis_off()

            # Title on first row
            if i == 0:
                ax.set_title(model_names[j], fontsize=12, pad=20)

            # Add mesh name on first column (but not for GT column)
            if j == 0 and False:  # Disabled text labels
                ax.text2D(
                    0.02,
                    0.5,
                    basenames[i].replace("_", "\n"),
                    transform=ax.transAxes,
                    rotation=90,
                    va="center",
                    ha="left",
                    fontsize=8,
                )

    def animate(frame):
        # Clear all axes
        for i in range(n_rows):
            for j in range(n_cols):
                axes[i, j].clear()
                ax = axes[i, j]

                # Reapply styling
                ax.grid(False)
                ax.xaxis.pane.fill = False
                ax.yaxis.pane.fill = False
                ax.zaxis.pane.fill = False
                for spine in (ax.xaxis, ax.yaxis, ax.zaxis):
                    spine.pane.set_edgecolor("none")
                ax.set_axis_off()

                # Title on first row
                if i == 0:
                    ax.set_title(model_names[j], fontsize=12, pad=20)

                # Add mesh name on first column (but not for GT column)
                if j == 0 and False:  # Disabled text labels
                    ax.text2D(
                        0.02,
                        0.5,
                        basenames[i].replace("_", "\n"),
                        transform=ax.transAxes,
                        rotation=90,
                        va="center",
                        ha="left",
                        fontsize=8,
                    )

                # Get mesh and plot
                mesh = meshes[(i, j)]

                # Calculate rotation angle
                angle = (frame / frames) * 360

                # Create depth-based coloring with enhanced visual appeal
                # Use Z-coordinate for depth mapping
                z_coords = mesh.vertices[:, 2]
                z_min, z_max = z_coords.min(), z_coords.max()

                # Normalize Z coordinates to 0-1 range for colormap
                if z_max > z_min:
                    z_normalized = (z_coords - z_min) / (z_max - z_min)
                else:
                    z_normalized = np.zeros_like(z_coords)

                # Use specified colormap (default viridis for nice blue-green-yellow)
                colormap = getattr(plt.cm, colormap_name, plt.cm.viridis)

                # Create the surface plot with Z-based coloring
                surf = ax.plot_trisurf(
                    mesh.vertices[:, 0],
                    mesh.vertices[:, 1],
                    mesh.vertices[:, 2],
                    triangles=mesh.faces,
                    cmap=colormap,
                    alpha=0.9,
                    linewidth=0,
                    edgecolor="none",
                )

                # Set the color values for each vertex
                surf.set_array(z_coords)

                # Set viewing angle
                ax.view_init(elev=20, azim=angle)

                # Set equal aspect & zoom
                set_axes_equal_and_zoom(ax, mesh, zoom=zoom)

        return []

    # Create animation
    print(f"Erstelle Animation mit {frames} Frames...")
    anim = FuncAnimation(fig, animate, frames=frames, interval=1000 // fps, blit=False)

    # Save as GIF mit spezifischer Auflösung
    print(f"Speichere GIF als '{output_file}' mit 960x720 Auflösung...")
    writer = PillowWriter(fps=fps)
    # Set DPI to ensure 960x720 resolution
    fig.set_dpi(75)
    anim.save(output_file, writer=writer, dpi=75)

    plt.close(fig)
    print(
        f"Animation gespeichert! Dateigröße: {os.path.getsize(output_file) / 1024 / 1024:.1f} MB"
    )

    return anim

In [4]:
# Mesh-Namen definieren
filenames = [
    "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
    "17803_Ox-eye_daisy_Leucanthemum_vulgare_pollen_grain",
    "21555_Hard_rush_Juncus_inflexus_pollen_grain_shrunken",
    "21188_Meadow_goats_beard_Tragopogon_pratensis_pollen_grain",
    "17878_Alder_Alnus_sp_pollen_grain_pentaporate",
]

## Methodenvergleich - Alle 5 besten Modelle
Vergleicht Ground Truth, Visual Hull, Pix2Vox, Hunyuan3D, Pixel2Mesh++ und PixelNeRF

In [ ]:
model_dirs_all = [
    "../data/processed/meshes",
    "../TestEvaluationPipeline/data/vh_2img",
    "../TestEvaluationPipeline/data/pix2vox_aug",
    "../TestEvaluationPipeline/data/Hunyuan3D-two-views",
    "../TestEvaluationPipeline/data/refine_p2mpp_augmentation_2_inputs",
    "../TestEvaluationPipeline/data/pollen_augmentation2",
]

model_names_all = [
    "GT",
    "Visual Hull",
    "Pix2Vox",
    "Hunyuan3D",
    "Pixel2Mesh++",
    "PixelNeRF",
]

# Erstelle große Übersichts-Animation (nur 3 Meshes für bessere Performance)
filenames_subset = [
    "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
    "17803_Ox-eye_daisy_Leucanthemum_vulgare_pollen_grain",
    "21188_Meadow_goats_beard_Tragopogon_pratensis_pollen_grain",
]

# MEMORY TEST: Nur 1 Mesh für minimalen Speicherverbrauch
filenames_test = [
    "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
]

print("🧪 MEMORY TEST: Teste mit nur 1 Mesh...")
create_animated_mesh_comparison(
    filenames_test,
    model_dirs_all,
    model_names_all,
    zoom=0.65,
    figsize=(10.0, 2.5),  # Etwas größer: 750x187 bei 75 DPI
    frames=6,  # Mehr Frames für smoothere Rotation (60° Schritte)
    fps=3,  # Langsamere Animation für bessere Sichtbarkeit
    output_file="test_single_mesh_hires.gif",
    colormap_name="plasma",
)

print("\n🎬 FULL COMPARISON: Alle 3 Meshes...")
create_animated_mesh_comparison(
    filenames_subset,
    model_dirs_all,
    model_names_all,
    zoom=0.65,
    figsize=(10.0, 7.5),  # Größer: 750x562 bei 75 DPI für mehr Details
    frames=6,  # 60° Schritte
    fps=3,  # Langsamere Animation
    output_file="all_methods_comparison_hires.gif",
    colormap_name="plasma",  # Beautiful purple-pink-yellow gradient
)

🧪 MEMORY TEST: Teste mit nur 1 Mesh...
Simplified mesh from 35 to 35 vertices
Simplified mesh from 35 to 35 vertices
Simplified mesh from 5090 to 5090 vertices
Erstelle Animation mit 4 Frames...
Speichere GIF als 'test_single_mesh.gif' mit 960x720 Auflösung...
Simplified mesh from 5090 to 5090 vertices
Erstelle Animation mit 4 Frames...
Speichere GIF als 'test_single_mesh.gif' mit 960x720 Auflösung...
Animation gespeichert! Dateigröße: 0.1 MB

🎬 FULL COMPARISON: Alle 3 Meshes...
Animation gespeichert! Dateigröße: 0.1 MB

🎬 FULL COMPARISON: Alle 3 Meshes...
Simplified mesh from 35 to 35 vertices
Simplified mesh from 35 to 35 vertices
Simplified mesh from 5090 to 5090 vertices
Simplified mesh from 5090 to 5090 vertices
Simplified mesh from 76 to 76 vertices
Simplified mesh from 76 to 76 vertices
Simplified mesh from 5644 to 5644 vertices
Simplified mesh from 5644 to 5644 vertices
Simplified mesh from 65 to 65 vertices
Simplified mesh from 65 to 65 vertices
Simplified mesh from 6728 to 67

KeyboardInterrupt: 

## Schnelle Single-Mesh Animation

In [ ]:
# Erstelle eine schnelle Animation für nur ein Mesh
single_mesh = ["17781_Common_thyme_Thymus_vulgaris_pollen_grain"]

create_animated_mesh_comparison(
    single_mesh,
    model_dirs_all,
    model_names_all,
    zoom=0.6,
    figsize=(12.8, 3.6),  # 960x270 für single row bei 75 DPI
    frames=48,  # Mehr Frames für smoothere Animation
    fps=12,  # Höhere FPS
    output_file="single_mesh_comparison.gif",
    colormap_name="coolwarm",  # Nice blue-white-red gradient
)

Erstelle Animation mit 48 Frames...
Speichere GIF als 'single_mesh_comparison.gif' mit 960x720 Auflösung...
Animation gespeichert! Dateigröße: 2.1 MB
Animation gespeichert! Dateigröße: 2.1 MB


In [ ]:
# Zeige die erstellten GIF-Dateien
import glob

gif_files = glob.glob("*.gif")
print("Erstellte GIF-Dateien:")
for gif in gif_files:
    size_mb = os.path.getsize(gif) / 1024 / 1024
    print(f"  {gif} - {size_mb:.1f} MB")

Erstellte GIF-Dateien:
  all_methods_comparison.gif - 4.2 MB
  pix2vox_comparison.gif - 10.2 MB
  pixelnerf_comparison.gif - 1.6 MB
  single_mesh_comparison.gif - 2.1 MB


In [ ]:
# High-resolution static visualization for Hunyuan3D
def create_static_hires_comparison(mesh_name, model_dirs, model_names, figsize=(20, 8)):
    """Create static high-resolution comparison showing full detail"""

    fig, axes = plt.subplots(
        2, len(model_dirs), figsize=figsize, subplot_kw={"projection": "3d"}
    )

    for j, (model_dir, model_name) in enumerate(zip(model_dirs, model_names)):
        # High-res view (top row)
        ax_hires = axes[0, j]
        # Simplified view (bottom row)
        ax_simple = axes[1, j]

        try:
            mesh_path = find_mesh_file(model_dir, mesh_name)
            if mesh_path:
                mesh = trimesh.load(mesh_path)

                # Original high-res mesh (top)
                ax_hires.plot_trisurf(
                    mesh.vertices[:, 0],
                    mesh.vertices[:, 1],
                    mesh.vertices[:, 2],
                    triangles=mesh.faces,
                    alpha=0.8,
                    cmap="plasma",
                )
                ax_hires.set_title(
                    f"{model_name}\nFull Resolution\n({mesh.vertices.shape[0]:,} vertices)",
                    fontsize=10,
                )

                # Simplified mesh (bottom) - show what animation uses
                if "hunyuan" in model_dir.lower() and mesh.vertices.shape[0] >= 20_000:
                    target_vertices = min(8_000, mesh.vertices.shape[0] // 2)
                    reduction_ratio = 1 - (target_vertices / mesh.vertices.shape[0])
                    reduction_ratio = max(0.2, min(0.7, reduction_ratio))
                elif mesh.vertices.shape[0] >= 5_000:
                    target_vertices = min(3_000, mesh.vertices.shape[0] // 3)
                    reduction_ratio = 1 - (target_vertices / mesh.vertices.shape[0])
                    reduction_ratio = max(0.5, min(0.95, reduction_ratio))
                else:
                    reduction_ratio = 0

                if reduction_ratio > 0:
                    new_verts, new_faces = fast_simplification.simplify(
                        points=mesh.vertices.view(np.ndarray),
                        triangles=mesh.faces.view(np.ndarray),
                        target_reduction=reduction_ratio,
                    )
                    simple_mesh = trimesh.Trimesh(vertices=new_verts, faces=new_faces)

                    ax_simple.plot_trisurf(
                        simple_mesh.vertices[:, 0],
                        simple_mesh.vertices[:, 1],
                        simple_mesh.vertices[:, 2],
                        triangles=simple_mesh.faces,
                        alpha=0.8,
                        cmap="viridis",
                    )
                    ax_simple.set_title(
                        f"Animation Version\n({simple_mesh.vertices.shape[0]:,} vertices)",
                        fontsize=10,
                    )
                else:
                    ax_simple.plot_trisurf(
                        mesh.vertices[:, 0],
                        mesh.vertices[:, 1],
                        mesh.vertices[:, 2],
                        triangles=mesh.faces,
                        alpha=0.8,
                        cmap="viridis",
                    )
                    ax_simple.set_title(
                        f"Animation Version\n(No simplification)", fontsize=10
                    )

                # Styling
                for ax in [ax_hires, ax_simple]:
                    ax.view_init(elev=20, azim=45)
                    ax.set_xlabel("X", fontsize=8)
                    ax.set_ylabel("Y", fontsize=8)
                    ax.set_zlabel("Z", fontsize=8)

        except Exception as e:
            for ax in [ax_hires, ax_simple]:
                ax.text(
                    0.5,
                    0.5,
                    0.5,
                    f"Error: {str(e)[:20]}...",
                    transform=ax.transAxes,
                    ha="center",
                    va="center",
                )

    plt.suptitle(
        f"High-Resolution vs Animation Comparison: {mesh_name}",
        fontsize=14,
        fontweight="bold",
    )
    plt.tight_layout()
    return fig


print("🔍 HIGH-RESOLUTION COMPARISON:")
print("Top row: Full resolution meshes")
print("Bottom row: Simplified versions used in animations")
print("=" * 60)

# Show high-res comparison for one mesh
test_mesh = "17781_Common_thyme_Thymus_vulgaris_pollen_grain"
hires_fig = create_static_hires_comparison(
    test_mesh, model_dirs_all, model_names_all, figsize=(24, 10)
)
plt.show()